<a href="https://colab.research.google.com/github/douglasmmachado/ExploratoryDataAnalysis/blob/master/Exploratory_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Data loading and visualization

We have a total of 75692 entries and 21 columns, but we can see that there is missing values in some columns. Also, categorical features such as ID_REF could be written as strings and not as objects for future data manipulations.

In [ ]:
raw_data_url ="https://raw.githubusercontent.com/douglasmmachado/ExploratoryDataAnalysis/master/project_database.csv"
df=pd.read_csv(raw_data_url, index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75692 entries, 0 to 75691
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID_REF            75689 non-null  object 
 1   ID_SITE_RATTACHE  75690 non-null  object 
 2   CODE_ATC          75685 non-null  object 
 3   HOSPI_CODE_UCD    75690 non-null  object 
 4   DATE_MOUV         75685 non-null  object 
 5   N_UFS             75692 non-null  int64  
 6   QUANTITY          75685 non-null  float64
 7   WEEK              75685 non-null  float64
 8   MONTH             75692 non-null  int64  
 9   YEAR              75685 non-null  float64
 10  N_ETB             75692 non-null  int64  
 11  POPULATION        75692 non-null  float64
 12  P_MEDICAL         75692 non-null  int64  
 13  PN_MEDICAL        75692 non-null  int64  
 14  LIT_HC            75685 non-null  float64
 15  LIT_HP            75692 non-null  float64
 16  SEJ_MCO           75692 non-null  int64 

<ipython-input-8-5fa22cf758ba>:2: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(raw_data_url, index_col=0)


In [ ]:
'''
From: Correlation analysis of factor impacting health
product consumption in French hospitals

12 main factors:
01 - Size population
02 - Number of physicians for consultations and hospitalization
03 - The number of beds
04 - The Number of medical visits or patient
05 - Facilities geographic position
06 - Type of health facilities
07 - Number of health facilities
08 - Seasonal factors
09 - Number of department or medical unit
10 - Number non-medical Staff (nurses, administration, …)
11 - Medical Specialist (health workers characteristics)
12 - Type of medical department


Table of attributes:
00 - ID_SITE - HOSPITAL_ID
01 - ID_REF - DRUG ID PER HOSPITAL
02 - DESIG - Drug Name
03 - CODE_UCD - Drug UCD code
04 - CODE_DEPT - Hospital district
05 - YEARS - Year of consumption
06 - MONTH - Month of consumption
07 - N_ETB - Numbers of hospital facilities
08 - N_UFS - Numbers of hospital medical units
09 - LIT_HC - Numbers of beds for full hospitalization
10 - LTT_HP - Numbers of beds for partial hospitalization
11 - SEJ_MCO - Numbers of visits in MCO departments
12 - SEJ_HAD - Numbers of visits in HAD departments
13 - SEJ_SLD - Numbers of visits in SLD departments
14 - SEJ_PSY - Numbers of visits in PSY departments
15 - PN_MEDICAL - Numbers of other staff (nurses; midwife)
16 - P_MEDICAL - Numbers of physician
17 - POPULATION - Number of population in the district

'''
df.sample(10)

,ID_REF,ID_SITE_RATTACHE,CODE_ATC,HOSPI_CODE_UCD,DATE_MOUV,N_UFS,QUANTITY,WEEK,MONTH,YEAR,...,POPULATION,P_MEDICAL,PN_MEDICAL,LIT_HC,LIT_HP,SEJ_MCO,SEJ_HAD,SEJ_PSY,SEJ_SSR,SEJ_SLD
154,960148,HOSPI_1,N02BE01,3400891996128,2014-03-24,9,1250.0,13.0,3,2014.0,...,571632.0,684,5321,1418.0,151.0,71879,0,0,1194,79
28999,9272958,HOSPI_2,N02AX02,3400892729589,2017-04-01,7,170.0,13.0,4,2017.0,...,539067.0,714,5001,1157.0,187.0,75420,0,1236,261,0
4895,800851,HOSPI_1,B05XA03,3400892761527,2018-10-09,6,380.0,41.0,10,2018.0,...,570662.0,805,5362,1364.0,101.0,81054,0,0,1203,93
9978,890264,HOSPI_1,N02AX02,3400892729589,2018-09-19,3,90.0,38.0,9,2018.0,...,570662.0,805,5362,1364.0,101.0,81054,0,0,1203,93
13660,841268,HOSPI_1,A03AX12,3400893736135,2015-10-12,5,140.0,42.0,10,2015.0,...,571879.0,684,5295,1411.0,94.0,74102,0,0,1140,57
18791,960687,HOSPI_1,N02AX02,3400892052120,2015-09-17,3,50.0,38.0,9,2015.0,...,571879.0,684,5295,1411.0,94.0,74102,0,0,1140,57
22769,9199612,HOSPI_2,N02BE01,3400891996128,2017-05-23,26,1060.0,21.0,5,2017.0,...,539067.0,714,5001,1157.0,187.0,75420,0,1236,261,0
38271,503129,HOSPI_3,N02BE01,3400891996128,2014-10-03,37,5500.0,40.0,10,2014.0,...,1107398.0,1158,7129,2063.0,521.0,117781,594,2903,1302,97
56404,504495,HOSPI_3,N05BA12,3400892075761,2015-12-24,6,90.0,52.0,12,2015.0,...,1120190.0,1239,7161,2053.0,493.0,118924,650,2878,1334,75
10042,890264,HOSPI_1,N02AX02,3400892729589,2018-12-04,12,410.0,49.0,12,2018.0,...,570662.0,805,5362,1364.0,101.0,81054,0,0,1203,93


## Data comprehension

In [ ]:
categorical_features = ['ID_REF', 'ID_SITE_RATTACHE', 'CODE_ATC', 'HOSPI_CODE_UCD']
date_features = ['DATE_MOUV', 'WEEK', 'MONTH', 'YEAR']
numerical_features = ['N_UFS', 'QUANTITY', 'N_ETB', 'POPULATION', 'P_MEDICAL', 'PN_MEDICAL', 'LIT_HC', 'LIT_HP', 'SEJ_MCO', 'SEJ_HAD', 'SEJ_PSY', 'SEJ_SSR'] 

In [19]:
for column in categorical_features:
  print(f'\n Unique values of feature: {column}')
  print(df[column].unique())
  print('\n')


 Unique values of feature: ID_REF
[960148 893880 960491 800851 820070 800852 890228 830701 890264 830303
 891086 840051 841268 830631 960229 890900 960388 960687 830302 822707
 810359 9199612 9122503 9387549 9276152 9266923 9276169 9206536 9083714
 9272958 9119122 9274584 9220364 9373613 9207576 '9207576' '9382670'
 '9269778' '9250856' '9205212' '9123520' '9208831' '9302263' nan 'X' 'Z'
 '00' '2345' '503129' '507146' '503386' '525441' '502400' '525443'
 '502369' '504394' '503182' '504200' '509929' '507071' '506197' '504495'
 '525057' '503095' '502829' '503184' '504403' '502374' '501463' 501463
 2463 5268 20395 7047 8704 7046 9490 3937 5186 20677 20420 8927 20845 2431
 5589 5066 5671 5193 2612 6180 21099]



 Unique values of feature: ID_SITE_RATTACHE
['HOSPI_1' 'HOSPI_2' nan 'HOSPI_3' 'HOSPI_4']



 Unique values of feature: CODE_ATC
['N02BE01' 'A12BA01' 'B05XA03' 'B01AB05' 'B01AC06' 'N05BA04' 'N02AX02'
 'N05CF01' 'R03AC02' 'H02AB06' 'A03AX12' 'N05BA12' 'B05XA09' 'M03BX01'
 'C01CA03' 

In [24]:
total = df.HOSPI_CODE_UCD.size
non_null = df.HOSPI_CODE_UCD.count()





75690